# Setup

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
%cd "/content/gdrive/MyDrive/PerfPred/Experiment 1/lang2vec"
!python3 setup.py install

/content/gdrive/.shortcut-targets-by-id/1vr6Z8seuUA0zoWaHuosCSZMv_H2Go5KR/PerfPred/Experiment 1/lang2vec
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_

In [3]:
from google.colab import auth
from google.auth import default
import gspread
import lang2vec.lang2vec as l2v
import numpy as np
import pandas as pd
import scipy
from pprint import pprint
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_distances

In [4]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Features

In [5]:
main_features = ['syntax_average', 'phonology_average', 'inventory_average',
               'syntax_knn', 'phonology_knn', 'inventory_knn', 'fam', 'geo']
extra_features = ["+".join(['syntax_average', 'phonology_average', 'inventory_average']),
                  "+".join(['syntax_knn', 'phonology_knn', 'inventory_knn']),
                  "+".join(['syntax_average', 'phonology_average', 'inventory_average', 'fam', 'geo']),
                  "+".join(['syntax_knn', 'phonology_knn', 'inventory_knn', 'fam', 'geo']),
                  "+".join(main_features)]
all_features = main_features + extra_features
N = len(all_features)

In [6]:
def feature_vecs(feats, langs, sheet):
  data = l2v.get_features(langs, '+'.join(feats), header=True)
  df = pd.DataFrame(data)
  worksheet = gc.open('l2v self-calculated distances').get_worksheet(sheet)
  worksheet.update_title('+'.join(feats))
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())
  return df

In [7]:
dfs = []
langs = ['kan', 'guj', 'hin', 'sin', 'tam']
for i, feat in enumerate(all_features):
  dfs.append(feature_vecs([feat], ["eng"] + langs, i))

KeyboardInterrupt: ignored

# Distances

In [161]:
def filter_df(df, langs):
  ret = df.copy()
  for lang in langs:
    ret = ret.loc[ret[lang] != '--']
  return ret

In [24]:
def distances(feats, langs, sheet1, sheet2, val_per_lang):
  cols = ["feature"] + langs
  dist_df = pd.DataFrame(columns=cols)
  num_df = pd.DataFrame(columns=cols)
  for i, feat in enumerate(feats):
    df = dfs[all_features.index(feat)]
    row = {"feature": feat}
    num_row = {"feature": feat}
    all_df = filter_df(df, ["eng"] + langs)
    for lang in langs:
      if val_per_lang:
        val_df = filter_df(df, ["eng", lang])
      else:
        val_df = all_df
      num_row[lang] = len(val_df)
      if len(val_df) == 0 or not np.any(val_df["eng"]) or not np.any(val_df[lang]):
        row[lang] = '--'
        continue
      val = scipy.spatial.distance.cosine(val_df["eng"], val_df[lang])
      row[lang] = np.round(val, decimals=4)
    dist_df.loc[len(dist_df.index)] = row
    num_df.loc[len(num_df.index)] = num_row
  worksheet1 = gc.open('l2v self-calculated distances').get_worksheet(sheet1)
  worksheet1.update([dist_df.columns.values.tolist()] + dist_df.values.tolist())
  worksheet2 = gc.open('l2v self-calculated distances').get_worksheet(sheet2)
  worksheet2.update([num_df.columns.values.tolist()] + num_df.values.tolist())
  return dist_df

In [164]:
langs = ['kan', 'guj', 'hin', 'sin', 'tam']
dist_df = distances(all_features, langs, N, N + 1, True)

In [165]:
langs = ['kan', 'guj', 'hin', 'sin', 'tam']
dist_df = distances(all_features, langs, N + 2, N + 3, False)

# Pre-calculated Distances

In [20]:
def pre_calc_distances(feats, langs, sheet):
  cols=["distance"] + langs
  dists = ['geographic', 'genetic', 'syntactic', 'phonological', 'inventory', 'featural']
  data = np.array(l2v.distance(dists, ['eng'] + langs))
  dist_df = pd.DataFrame(columns=cols)
  for i, dist in enumerate(dists):
    vals = data[i,0,1:]
    row = dict(zip(cols, [dist] + list(vals)))
    dist_df.loc[len(dist_df.index)] = row
  worksheet = gc.open('l2v self-calculated distances').get_worksheet(sheet)
  worksheet.update([dist_df.columns.values.tolist()] + dist_df.values.tolist())
  return dist_df

In [21]:
langs = ['kan', 'guj', 'hin', 'sin', 'tam']
dist_df = pre_calc_distances(all_features, langs, N + 5)

# Main Sheet

In [31]:
def distance_with_eng_main(feats, langs, sheet):
  cols = ["feature"] + [lang + "-eng" for lang in langs]
  dist_df = pd.DataFrame(columns=cols)
  for i, feat in enumerate(feats):
    df = dfs[all_features.index(feat)]
    row = {"feature": feat}
    for lang in langs:
      val = scipy.spatial.distance.cosine(df["eng"], df[lang])
      row[lang + "-eng"] = np.round(val, decimals=4)
    dist_df.loc[len(dist_df.index)] = row
  worksheet = gc.open('Experiment 1 Data').get_worksheet(sheet)
  worksheet.update([dist_df.columns.values.tolist()] + dist_df.values.tolist())
  return dist_df

In [32]:
langs = ['kan', 'guj', 'hin', 'sin', 'tam']
select_features = ['syntax_knn', 'phonology_knn', 'inventory_knn', 'fam', 'geo',
                  "+".join(['syntax_knn', 'phonology_knn', 'inventory_knn']),
                  "+".join(['syntax_knn', 'phonology_knn', 'inventory_knn', 'fam', 'geo'])]
dist_df = distance_with_eng_main(select_features, langs, 3)